## Solve using Benders

In [1]:
from gurobipy import *

In [2]:
import qminos
from qminos import qwarmLP

In [3]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

In [5]:
from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders

In [6]:
from cobra.io import load_json_model

ijomc  = load_json_model('/home/laurence/ME/data/dynamicME/nominal_ijomc.json')
ijofit = load_json_model('/home/laurence/ME/data/dynamicME/best_ijomc.json')

In [7]:
from dynamicme.optimize import Optimizer

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fa46f723cf8>)


In [8]:
opt = Optimizer(ijomc)

In [9]:
gap = opt.add_duality_gap_constraint()

In [10]:
crowding = ijomc.metabolites.crowding
crowdingfit = ijofit.metabolites.crowding
a12_dict = {(gap.metabolites.get_by_id(crowding.id), gap.reactions.get_by_id(rxn.id)):(
    rxn.metabolites[crowding], 
    ijofit.reactions.get_by_id(rxn.id).metabolites[crowdingfit]) for rxn in crowding.reactions}

In [11]:
opt.make_disjunctive_primal_dual(gap, a12_dict)

<Model duality_gap at 0x7fa47108fad0>

In [12]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary_'):
    rxn.objective_coefficient = 1.

In [13]:
ijofit.optimize()
MU_FIT = ijofit.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

In [83]:
MU_FIT

0.7301837182736599

In [84]:
F_TOL = 0.1

gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = MU_FIT*(1+F_TOL)
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = MU_FIT*(1-F_TOL)

In [85]:
gap.optimize(solver='gurobi', objective_sense='minimize')

<Solution 1.00 at 0x7fa46915d550>

In [86]:
sum([r.x for r in gap.reactions.query('binary_')])

1.0

In [87]:
bvars = [r for r in gap.reactions if r.variable_kind == 'integer']
len(bvars)

2018

In [88]:
gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x

0.7489994413180718

In [89]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        print rxn, rxn.x

binary_crowding_ATPS4rpp_abs 1.0


## Solve using Benders

In [90]:
from cobra.solvers import gurobi_solver
from gurobipy import *

In [91]:
%%time
milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

CPU times: user 464 ms, sys: 17.4 ms, total: 482 ms
Wall time: 456 ms


In [92]:
milp.Params.IntFeasTol = 1e-9
milp.Params.FeasibilityTol = 1e-8
milp.Params.OptimalityTol = 1e-8
milp.Params.NumericFocus = 1

In [93]:
%time milp.optimize()

CPU times: user 9.6 s, sys: 293 ms, total: 9.9 s
Wall time: 3.69 s


In [94]:
milp.ObjVal

1.0

In [95]:
sol = gurobi_solver.format_solution(milp, gap)

In [96]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xmilp = sol.x_dict[rxn.id]
        print rxn, rxn.x, xmilp

binary_crowding_ATPS4rpp_abs 1.0 1.0


### Double check that master, sub split correctly

In [97]:
# ### Any constraints involving only y?
# bin_vars = gap.reactions.query('binary_')
# cons_bin = set([m for bv in bin_vars for m in bv.metabolites.keys()])
# len(cons_bin)
# cons_vtypes = [set([r.variable_kind for r in m.reactions]) for m in cons_bin]
# cons_bin_only = [c for c in cons_vtypes if 'continuous' not in c]
# cons_bin_only

## Subproblem infeasible...?

In [98]:
decomposer = Decomposer(milp)

master,sub = decomposer.benders_decomp()

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


### Try solving sub in quad

In [99]:
from qminos import quadLP
import numpy as np

yopt0 = [y.X for y in decomposer._y0]
decomposer.update_subobj(yopt0)
A = decomposer._A
d = decomposer._d
c = np.array(decomposer._cx)
xl = decomposer._xl
xu = decomposer._xu
csense = decomposer._csenses

In [ ]:
from qminos.quadLP import QMINOS

solver = QMINOS()
%time qx,inform,hs = solver.solvelp(A,d,c,xl,xu,csense, precision='quad')
print(inform)

In [79]:
from qminos.quadLP import QMINOS

solver = QMINOS()
%time qx,inform,hs = solver.solvelp(A,d,c,xl,xu,csense, precision='dq')
print(inform)

Getting MINOS parameters...
Done in 10.0109 seconds with status 1
CPU times: user 10.1 s, sys: 2.76 ms, total: 10.1 s
Wall time: 10.1 s
1


In [78]:
inform

array(1)

In [43]:
yopt0 = [y.X for y in decomposer._y0]
decomposer.update_subobj(yopt0)
sub.Params.OutputFlag = 1
sub.Params.NumericFocus = 1
sub.Params.BarHomogeneous = 1
# sub.Params.FeasibilityTol = 1e-9
# sub.Params.OptimalityTol = 1e-9
sub.optimize()

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter NumericFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter BarHomogeneous to 1
   Prev: -1  Min: -1  Max: 1  Default: -1
Optimize a model with 16506 rows, 55295 columns and 100457 nonzeros
Coefficient statistics:
  Matrix range     [2e-11, 1e+03]
  Objective range  [5e-03, 1e+04]
  Bounds range     [1e+03, 1e+03]
  RHS range        [0e+00, 0e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 4864 rows and 30399 columns
Presolve time: 0.06s
Presolved: 11642 rows, 24896 columns, 61914 nonzeros

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 19

### Solve subproblem using qMINOS to overcome numerical issues

In [70]:
import qminos.quadLP

ImportError: libgfortran.so.3: cannot open shared object file: No such file or directory

In [67]:
for rxn in gap.reactions:
    for met,s in rxn.metabolites.iteritems():
        if abs(s)>0 and abs(s)<1e-10:
            print '%-20.3g%-25.20s%-25.20s'% (s, rxn.id, met.id)

3.42e-11            APG3PAT181               crowding                 
4.61e-11            CDAPPA181                crowding                 
2.27e-11            DURIK1                   crowding                 
6.43e-11            FACOAL161t2pp            crowding                 
5.86e-11            FE3abcpp                 crowding                 
4.21e-11            FTHFD                    crowding                 
2.94e-11            HDCEAtexi                crowding                 
6.87e-11            IPMD                     crowding                 
3.53e-11            PRAIS                    crowding                 
7.17e-11            SBTptspp                 crowding                 
9.78e-11            FUCtex_abs               crowding                 
4.29e-11            GLYAT_abs                crowding                 
5.09e-11            THRPtex_abs              crowding                 
7.69e-11            TKT2_abs                 crowding                 
1.96e-

In [60]:
A = decomposer._A 
print(abs(A[A!=0]).min())

B = decomposer._B
abs(B[B!=0]).min()

1.95717576106e-11


1000.0

In [30]:
xdict = {x0.VarName: sub.getConstrByName(x0.VarName).Pi for x0 in decomposer._x0}

In [31]:
import numpy as np
errs = np.array([xdict[x0.VarName]-x0.X for x0 in decomposer._x0])
sum(abs(errs))

594653.06191386702

In [32]:
mu_ind = gap.reactions.index(gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M)
print('cobra: %g, MILP: %g, Benders: %g' % (gap.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x, milp.getVarByName(str(mu_ind)).X, xdict[str(mu_ind)]))

cobra: 0.740682, MILP: 0.740682, Benders: 0.740778


In [33]:
decomposer = Decomposer(milp)

master,sub = decomposer.benders_decomp()
master.Params.IntFeasTol = 1e-9
#master.Params.FeasibilityTol = 1e-5 #milp.Params.FeasibilityTol
#master.Params.OptimalityTol = 1e-5 #milp.Params.OptimalityTol
master.Params.NumericFocus = 1
#master.Params.MIPGap = 1e-6
master.Params.OutputFlag = 1
master.Params.Presolve = 0#-1
master._verbosity = 1

sub.Params.Presolve = 0#-1
sub.Params.OutputFlag = 0
sub.Params.NumericFocus = 1
sub.Params.BarHomogeneous = 1
#sub.Params.FeasibilityTol = 1e-5 #milp.Params.FeasibilityTol
#sub.Params.OptimalityTol = 1e-5 #milp.Params.OptimalityTol
sub.Params.NumericFocus = 1
%time master.optimize(cb_benders)

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Parameter IntFeasTol unchanged
   Value: 1e-09  Min: 1e-09  Max: 0.1  Default: 1e-05
Changed value of parameter NumericFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Parameter Presolve unchanged
   Value: 0  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 1 rows, 2019 columns and 2019 nonzeros
Variable types: 1 continuous, 2018 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
zmaster=0. zsub=4.0964. gap=-4.0964
Variable types: 1 continu

In [35]:
yopt = [y.X for y in decomposer._ys]
print('sum(yopt): %g' % sum(yopt))
decomposer.update_subobj(yopt)
sub.optimize()

sum(yopt): 0


In [36]:
master.ObjVal

0.0

In [37]:
master.getVarByName('z').X

0.0

In [38]:
xdict = {x0.VarName: sub.getConstrByName(x0.VarName).Pi for x0 in decomposer._x0}

In [39]:
xdict['7']

0.7447873926391331

In [40]:
milp.getVarByName('7')

<gurobi.Var 7 (value 0.740681972939)>

In [41]:
sol.x_dict[gap.reactions[7].id]

0.740681972939253

In [30]:
from gurobipy import *

In [31]:
master.Status == GRB.Status.OPTIMAL

True

In [32]:
milp.ObjVal

2017.9999999656293

In [33]:
master.ObjVal

0.0

In [34]:
sub.ObjVal

3.702479850131916

In [35]:
sub.Status

2

In [36]:
sum(yopt)

0.0

In [37]:
import numpy as np
#errs = [xdict[x0.VarName]-x0.X for x0 in milp.getVars() ]
errs = np.array([xdict[x0.VarName]-x0.X for x0 in decomposer._x0])
sum(abs(errs))

994828.93362923583

In [38]:
for rxn in gap.reactions.query('binary_'):
    if rxn.x > 0.99:
        xbend = sub.getConstrByName(rxn.id).Pi
        print rxn, rxn.x, xmilp

AttributeError: 'NoneType' object has no attribute 'Pi'